Yes, I know it's sloppy.  

In [ ]:
import requests
import re 
import tabula
import pandas as pd
import urllib 
import time
import numpy 
import sqlite3

# Read pdf into list of DataFrame
partial_tables = tabula.read_pdf("krausebooklist.pdf", pages='all')
df = pd.concat(partial_tables)

In [ ]:
def from_search_results(resp):
    if resp.json().get('numFound'):
        doc = resp.json()['docs'][0]
        return [doc['key'], doc.get('isbn', [None])[0], doc.get('first_sentence'), doc.get('subject')]
    return ['', '', [], []]

def lookup(row):
    try:
        tries_remaining = 5
        while tries_remaining > 0:
            time.sleep(3)
            title = row['Title'].split(":")[0]
            author = row['Author'].split(",")[0]
            solr_qry = urllib.parse.quote(f"{title} {author}")
            url = f"http://openlibrary.org/search.json?q={solr_qry}"
            print(url)
            resp = requests.get(url)
            # print(f"{resp.ok} {resp.status_code} {resp.reason}")
            if resp.ok:
                result = from_search_results(resp)
                return result
            if resp.status_code >= 500:
                time.sleep(10)
                tries_remaining -= 1 
            else:
                break
        return ['', '', [], []]
    except Exception as exc:
        print(exc)
        return ['', '', [], []]

df['ol_info'] = df.apply(lookup, axis=1)

In [ ]:
field_names = ('ol_link', 'isbn', 'first_sentence', 'subjects')
for (idx, field_name) in enumerate(field_names):
    df[field_name] = df.ol_info.apply(lambda lst: lst[idx])

In [ ]:
df['id'] = numpy.arange(len(df))

In [ ]:
subjects = []
def extract_subjects(row):
    for subj in (row.subjects or []):
        subjects.append((row.id, subj))
df.apply(extract_subjects, axis=1)        

In [ ]:
subjects_df = pd.DataFrame(subjects, columns=['book_id', 'subject'])

In [ ]:
df['first_sentence_clean'] = df.first_sentence.apply(lambda s: s[0] if s else '')

In [ ]:
fields = ['id', 'Title', 'Author', 'Published', 'ol_link', 'isbn', 'first_sentence_clean', ]

In [ ]:
final = df[fields]

In [ ]:
final = final.rename({'first_sentence_clean': 'first_sentence'}, axis=1)

In [ ]:
final = final.set_index('id')

In [ ]:
conn = sqlite3.connect('texas-targeted-books.db')
final.to_sql('book', conn, if_exists='replace', index=True)

In [ ]:
subjects_df.to_sql('subject', conn, if_exists='replace', index=False)

In [ ]:
def title(row):
    if row.ol_link:
        return f'<a href="https://openlibrary.org{row.ol_link}">{row.Title}</a>'
    else:
        return row.Title

In [ ]:
def row_string(row):
    return(f"""<tr><td>{title(row)}</td><td>{row.Author}</td>
        <td>{row.Published}</td><td>{row.isbn}</td>    
        <td>{', '.join(row.subjects or [])}</td>
        </tr>
        """)

In [ ]:
rows = "\n".join(df.sort_values(['Title', 'Published']).apply(row_string, axis=1))

In [ ]:
with open("README.md", "w") as outfile:
    outfile.write(f"""
# Texas-Targeted Books

The [Texas Tribune reported](https://www.texastribune.org/2021/10/26/texas-school-books-race-sexuality/)
on Oct. 26, 2021 that Texas State Rep. Matt Krause,
in his role as chair of the Texas House Committee on General Investigating,
opened an official investigation into the presence of hundreds of books
in Texas schools, and has required all Texas schools to report on the
                books' presence.

Here is a best-effort presentation of the Rep. Krause list
in RDBMS and HTML table form, with hyperlinks. ISBN numbers,
subjects, and links to OpenLibrary
entries were made by automated query to the OpenLibrary API;
no guarantees on their accuracy.

- [List in relational database format](texas-targeted-books.db) ([SQLite](https://sqlite.org/index.html))
- [Chair Krause's letter](https://static.texastribune.org/media/files/965725d7f01b8a25ca44b6fde2f5519b/krauseletter.pdf)
- [Chair Krause's list](https://static.texastribune.org/media/files/94fee7ff93eff9609f141433e41f8ae1/krausebooklist.pdf) (PDF)
- [Code generating this page](query_open_library.ipynb)

<table>
  <tr><td>Title</td><td>Author</td><td>Year</td><td>ISBN</td><td>Subjects</td></tr>
  {rows}
</table>
""")